In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
from keras import layers
import os
import matplotlib.pyplot as plt
#from imblearn.under_sampling import RandomUnderSampler
#from tabulate import tabulate
from itertools import islice
from scipy.signal import resample
from sklearn.preprocessing import StandardScaler
import pickle
import glob
import random
#import dask.dataframe as dd

In [3]:
data_path = '/content/drive/MyDrive/MovePort/'
#Modified_Df_PATH = '/content/drive/MyDrive/MovePortModif/'
participants = sorted([p for p in os.listdir(data_path) if p.isdigit()], key=int)
print("Participants disponibles :", participants)
activities = ['back', 'forward', 'halfsquat', 'still']
modalities = ['ips_1', 'emg_1', 'imu_1']

Participants disponibles : ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25']


In [8]:
def list_missing_files(base_path, participants, activities, modalities):

  missing_files = {}
  for participant in participants:
    participant_missing = {}
    for activity in activities:
      for modality in modalities:
        file_path = os.path.join(base_path, str(participant), activity, f"{modality}.csv")
        if not os.path.exists(file_path):
          if activity not in participant_missing:
            participant_missing[activity] = []
          participant_missing[activity].append(modality)
    if participant_missing:
      missing_files[participant] = participant_missing
  return missing_files

In [ ]:
# Lister les fichiers manquants
missing_files = list_missing_files(data_path, participants, activities, modalities)
# Afficher les résultats
participants_manque_files = []
print("Participants avec des fichiers de modalités manquants :")
for participant, activities in missing_files.items():
    print(f"Participant {participant}:")
    participants_manque_files.append(participant)
    for activity, modalities in activities.items():
        print(f"  Activité {activity}: Modalités manquantes: {', '.join(modalities)}")

Participants avec des fichiers de modalités manquants :
Participant 18:
  Activité forward: Modalités manquantes: ips_1, emg_1, imu_1
Participant 19:
  Activité forward: Modalités manquantes: ips_1, emg_1, imu_1
Participant 20:
  Activité forward: Modalités manquantes: ips_1, emg_1, imu_1
Participant 21:
  Activité forward: Modalités manquantes: ips_1, emg_1, imu_1
Participant 22:
  Activité back: Modalités manquantes: ips_1, emg_1, imu_1
Participant 23:
  Activité forward: Modalités manquantes: ips_1, emg_1, imu_1
  Activité halfsquat: Modalités manquantes: ips_1, emg_1, imu_1
Participant 24:
  Activité halfsquat: Modalités manquantes: ips_1, emg_1, imu_1


In [ ]:
#participants_manque_files

In [ ]:
"""def get_column_structure_and_shape(base_path, participants, activities, modalities):

  column_structure = {}
  data_shapes = {}
  for modality in modalities:
      for participant in participants:
          for activity in activities:
              file_path = os.path.join(base_path, str(participant), activity, f"{modality}.csv")
              if os.path.exists(file_path):
                  df = pd.read_csv(file_path, header=0)
                  column_structure[modality] = df.columns
                  data_shapes[modality] = df.shape
                  break
          if modality in column_structure:
              break
      if modality not in column_structure:
          raise ValueError(f"Structure des colonnes introuvable pour la modalité {modality}")

  return column_structure, data_shapes"""

'def get_column_structure_and_shape(base_path, participants, activities, modalities):\n\n  column_structure = {}\n  data_shapes = {}\n  for modality in modalities:\n      for participant in participants:\n          for activity in activities:\n              file_path = os.path.join(base_path, str(participant), activity, f"{modality}.csv")\n              if os.path.exists(file_path):\n                  df = pd.read_csv(file_path, header=0)\n                  column_structure[modality] = df.columns\n                  data_shapes[modality] = df.shape\n                  break\n          if modality in column_structure:\n              break\n      if modality not in column_structure:\n          raise ValueError(f"Structure des colonnes introuvable pour la modalité {modality}")\n\n  return column_structure, data_shapes'

In [ ]:
"""def calculate_mean_structure(base_path, participants, activities, modalities):

  mean_data = {modality: [] for modality in modalities}
  for activity in activities:
      for modality in modalities:
          participant_pairs = random.sample(participants, 2)
          modality_data = []
          for participant in participant_pairs:
              file_path = os.path.join(base_path, str(participant), activity, f"{modality}.csv")
              if os.path.exists(file_path):
                  try:
                      df = pd.read_csv(file_path, header=0)
                      numeric_df = df.apply(pd.to_numeric, errors='coerce')  # Convertir en numérique
                      modality_data.append(numeric_df)
                  except Exception as e:
                      print(f"Erreur lors du chargement de {file_path}: {e}")
          if modality_data:
              combined_df = pd.concat(modality_data)
              combined_df = combined_df.apply(pd.to_numeric, errors='coerce')
              combined_df = combined_df.dropna(axis=1, how='all')  # Supprimer les colonnes entièrement non numériques
              mean_df = combined_df.mean(axis=0).to_frame().transpose()  # Calculer uniquement les colonnes numériques
              mean_df = round(mean_df, 3)
              mean_data[modality].append(mean_df)

  for modality in modalities:
      mean_data[modality] = pd.concat(mean_data[modality], axis=0).mean(axis=0).to_frame().transpose()

  return mean_data"""

'def calculate_mean_structure(base_path, participants, activities, modalities):\n\n  mean_data = {modality: [] for modality in modalities}\n  for activity in activities:\n      for modality in modalities:\n          participant_pairs = random.sample(participants, 2)\n          modality_data = []\n          for participant in participant_pairs:\n              file_path = os.path.join(base_path, str(participant), activity, f"{modality}.csv")\n              if os.path.exists(file_path):\n                  try:\n                      df = pd.read_csv(file_path, header=0)\n                      numeric_df = df.apply(pd.to_numeric, errors=\'coerce\')  # Convertir en numérique\n                      modality_data.append(numeric_df)\n                  except Exception as e:\n                      print(f"Erreur lors du chargement de {file_path}: {e}")\n          if modality_data:\n              combined_df = pd.concat(modality_data)\n              combined_df = combined_df.apply(pd.to_numeric,

In [ ]:
"""def create_missing_files_with_mean(base_path, participant_id, activities, modalities, mean_data, column_structure, data_shapes):

  participant_path = os.path.join(base_path, str(participant_id))
  for activity in activities:
      activity_path = os.path.join(participant_path, activity)
      if not os.path.exists(activity_path):
          os.makedirs(activity_path)  # Crée le dossier d'activité s'il n'existe pas
      for modality in modalities:
          file_path = os.path.join(activity_path, f"{modality}.csv")
          if not os.path.exists(file_path):
              print(f"Création de {file_path} avec des données moyennes et structure correcte")
              shape = data_shapes[modality]
              synthetic_data = pd.DataFrame(np.tile(mean_data[modality].values, (shape[0], 1)), columns=mean_data[modality].columns)
              synthetic_data = synthetic_data.reindex(columns=column_structure[modality], fill_value=0)  # Alignement des colonnes
              synthetic_data.to_csv(file_path, index=False)"""

'def create_missing_files_with_mean(base_path, participant_id, activities, modalities, mean_data, column_structure, data_shapes):\n\n  participant_path = os.path.join(base_path, str(participant_id))\n  for activity in activities:\n      activity_path = os.path.join(participant_path, activity)\n      if not os.path.exists(activity_path):\n          os.makedirs(activity_path)  # Crée le dossier d\'activité s\'il n\'existe pas\n      for modality in modalities:\n          file_path = os.path.join(activity_path, f"{modality}.csv")\n          if not os.path.exists(file_path):\n              print(f"Création de {file_path} avec des données moyennes et structure correcte")\n              shape = data_shapes[modality]\n              synthetic_data = pd.DataFrame(np.tile(mean_data[modality].values, (shape[0], 1)), columns=mean_data[modality].columns)\n              synthetic_data = synthetic_data.reindex(columns=column_structure[modality], fill_value=0)  # Alignement des colonnes\n             

In [ ]:
# Calculer les moyennes à partir des participants 1 à 16
existing_participants = list(range(1, 17))

In [ ]:
# Créer les fichiers manquants pour chaque participant de la liste
#column_structure, data_shapes = get_column_structure_and_shape(data_path, existing_participants, activities, modalities)
#for participant_id in participants_manque_files:
    #mean_data = calculate_mean_structure(data_path, existing_participants, activities, modalities)
    #create_missing_files_with_mean(data_path, participant_id, activities, modalities, mean_data, column_structure, data_shapes)

print("Tous les dossiers et fichiers manquants ont été créés avec des valeurs moyennes et une structure correcte.")

Tous les dossiers et fichiers manquants ont été créés avec des valeurs moyennes et une structure correcte.


In [ ]:
"""newFileips = pd.read_csv('/content/drive/MyDrive/MovePort/18/forward/ips_1.csv')
newFileemg = pd.read_csv('/content/drive/MyDrive/MovePort/19/forward/emg_1.csv')
newFileimu = pd.read_csv('/content/drive/MyDrive/MovePort/21/forward/imu_1.csv')
print('shape of ips' ,newFileips.shape)
print('shape of emg' ,newFileemg.shape)
print('shape of imu' ,newFileimu.shape)"""

"newFileips = pd.read_csv('/content/drive/MyDrive/MovePort/18/forward/ips_1.csv')\nnewFileemg = pd.read_csv('/content/drive/MyDrive/MovePort/19/forward/emg_1.csv')\nnewFileimu = pd.read_csv('/content/drive/MyDrive/MovePort/21/forward/imu_1.csv')\nprint('shape of ips' ,newFileips.shape)\nprint('shape of emg' ,newFileemg.shape)\nprint('shape of imu' ,newFileimu.shape)"

In [ ]:
# justqu'au la c'est carré !

In [34]:
from sklearn.preprocessing import LabelEncoder
# Encoder les catégories
categories = ['back', 'forward', 'halfsquat', 'still']
category_encoder = LabelEncoder()
category_encoder.fit(categories)

LabelEncoder()

In [35]:
def decode_categories(df, column_name='categorie'):
  df[column_name] = category_encoder.inverse_transform(df[column_name])
  return df

In [36]:
def inverser_csv(file_path):
  df = pd.read_csv(file_path, header=0)
  df = df.transpose()
  df.columns = df.iloc[0]  # Première ligne devient l'en-tête
  df = df[1:].reset_index(drop=True)  # Supprimer la première ligne et réindexer
  return df

In [38]:
def process_data_in_batch(participant_path, modality):
  # Dictionnaire pour mapper les catégories aux valeurs numériques manuellement
  # category_mapping = {'back': 0, 'forward': 1, 'halfsquat': 2, 'still': 3}
  all_category_data = []

  for category in categories:
    # Chercher les fichiers spécifiques à la modalité
    files = glob.glob(os.path.join(participant_path, category, f"{modality}_1.csv"))
    print(f"  Fichiers {modality.upper()} trouvés pour la catégorie '{category}': {files}")
    category_data = []  # Liste pour stocker les données de cette catégorie
    for file in files:
      print(f"    Traitement du fichier {modality.upper()} : {file}")
      try:
        df = inverser_csv(file)
        df['categorie'] = category_encoder.transform([category])[0]
        category_data.append(df)
      except Exception as e:
        print(f"Erreur lors du traitement du fichier {file}: {e}")
    # Concaténer toutes les données de la catégorie pour ce participant
    if category_data:
      concatenated_category_data = pd.concat(category_data, ignore_index=True)
      all_category_data.append(concatenated_category_data)
  # Retourner toutes les données des catégories pour ce participant
  if all_category_data:
    return pd.concat(all_category_data, ignore_index=True)
  else:
    return None

In [39]:
def process_participants(data_path, participants, modality, batch_size=5):

  all_data = []  # Liste pour stocker toutes les données traitées
  # Diviser les participants en lots (batchs)
  participants_batches = [participants[i:i + batch_size] for i in range(0, len(participants), batch_size)]

  for batch_idx, batch_participants in enumerate(participants_batches, start=1):
    print(f"Traitement du lot {batch_idx}/{len(participants_batches)} : Participants {batch_participants}")
    batch_data = []  # Liste pour stocker les données d'un batch
    for participant_id in batch_participants:
      participant_path = os.path.join(data_path, str(participant_id))
      # Traiter les données pour ce participant
      print(f"Participant {participant_id}")
      participant_data = process_data_in_batch(participant_path, modality)
      # Ajouter l'ID du participant aux données
      if participant_data is not None:
        participant_data['id'] = participant_id  # Ajouter l'ID du participant
        batch_data.append(participant_data)
    # Concaténer les données du lot
    if batch_data:
      batch_df = pd.concat(batch_data, ignore_index=True)
      all_data.append(batch_df)
  # Concaténer toutes les données traitées en un seul DataFrame
  if all_data:
    concatenated_data = pd.concat(all_data, ignore_index=True)
    return concatenated_data
  else:
    print("Aucune donnée valide trouvée.")
    return None

In [40]:
batch_size = 5  # regroupe les participants en 5
#participants = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
participants = participants

In [41]:
modalityImu = 'imu'  # Modalité, ici imu
modalityEmg = 'emg'  # Modalité, ici emg
modalityIps = 'ips'  # Modalité, ici ips
# Appel de la fonction pour traiter les données
processed_dataImu = process_participants(data_path, participants, modalityImu, batch_size)
processed_dataEmg = process_participants(data_path, participants, modalityEmg, batch_size)
processed_dataIps = process_participants(data_path, participants, modalityIps, batch_size)

Traitement du lot 1/5 : Participants ['1', '2', '3', '4', '5']
Participant 1
  Fichiers IMU trouvés pour la catégorie 'back': ['/content/drive/MyDrive/MovePort/1/back/imu_1.csv']
    Traitement du fichier IMU : /content/drive/MyDrive/MovePort/1/back/imu_1.csv
  Fichiers IMU trouvés pour la catégorie 'forward': ['/content/drive/MyDrive/MovePort/1/forward/imu_1.csv']
    Traitement du fichier IMU : /content/drive/MyDrive/MovePort/1/forward/imu_1.csv
  Fichiers IMU trouvés pour la catégorie 'halfsquat': ['/content/drive/MyDrive/MovePort/1/halfsquat/imu_1.csv']
    Traitement du fichier IMU : /content/drive/MyDrive/MovePort/1/halfsquat/imu_1.csv
  Fichiers IMU trouvés pour la catégorie 'still': ['/content/drive/MyDrive/MovePort/1/still/imu_1.csv']
    Traitement du fichier IMU : /content/drive/MyDrive/MovePort/1/still/imu_1.csv
Participant 2
  Fichiers IMU trouvés pour la catégorie 'back': ['/content/drive/MyDrive/MovePort/2/back/imu_1.csv']
    Traitement du fichier IMU : /content/drive/M

In [42]:
print("imu procces shape : ", processed_dataImu.shape)
print("emg procces shape : ", processed_dataEmg.shape)
print("ips procces shape : ", processed_dataIps.shape)

imu procces shape :  (551504, 56)
emg procces shape :  (11029891, 18)
ips procces shape :  (343310, 684)


In [ ]:
"""print("imu procces nulls : ", processed_dataImu.isnull().sum())
print("emg procces nulls : ", processed_dataEmg.isnull().sum())
print("ips procces nulls : ", processed_dataIps.isnull().sum())"""

'print("imu procces nulls : ", processed_dataImu.isnull().sum())\nprint("emg procces nulls : ", processed_dataEmg.isnull().sum())\nprint("ips procces nulls : ", processed_dataIps.isnull().sum())'

In [ ]:
processed_dataImu.head(3)

Unnamed: 0,Head_Acc_X,Head_Acc_Y,Head_Acc_Z,Head_Gyr_X,Head_Gyr_Y,Head_Gyr_Z,Head_Roll,Head_Pitch,Head_Yaw,Waist_Acc_X,...,R_F_Acc_Y,R_F_Acc_Z,R_F_Gyr_X,R_F_Gyr_Y,R_F_Gyr_Z,R_F_Roll,R_F_Pitch,R_F_Yaw,categorie,id
0,8.902975,0.412529,4.052022,-0.015015,0.01026,-0.009929,-12.639535,-10.234849,30.077293,9.578385,...,-1.903201,9.019931,-0.027493,0.009596,-0.008538,10.781965,35.531989,-17.776944,0,1
1,8.912331,0.391603,4.045956,-0.010874,0.014445,-0.006227,-12.643968,-10.226859,30.069993,9.540294,...,-1.942165,9.052294,-0.016795,0.007321,-0.009379,10.773012,35.535744,-17.793866,0,1
2,8.88999,0.384217,4.039017,-0.013686,0.01211,-0.005442,-12.647921,-10.220573,30.063655,9.561864,...,-1.965231,9.028006,-0.009752,0.004455,-0.011313,10.76855,35.536932,-17.806589,0,1


In [ ]:
processed_dataEmg.head(3)

Unnamed: 0,R_Vlat,R_RF,R_ST,R_TA,L_Vlat,L_RF,L_ST,L_TA,R_MG,R_LG,R_SOL,R_IL,L_MG,L_LG,L_SOL,L_IL,categorie,id
0,-0.704956,1.309204,-2.316284,-20.24231,-6.747437,2.920532,-1.208496,-6.948853,-0.906372,-12.387085,54.180908,-2.5177,-0.704956,10.876465,-3.52478,-3.52478,0,1
1,2.01416,1.309204,-2.316284,-16.415405,-1.208496,2.618408,-1.208496,-6.948853,-0.906372,-12.387085,54.180908,-2.5177,-0.704956,10.876465,-3.52478,-3.52478,0,1
2,-0.704956,1.309204,-2.316284,-16.314697,-1.107788,1.107788,-1.208496,-6.948853,-0.906372,-12.387085,54.180908,-2.5177,-0.704956,10.876465,-3.52478,-3.52478,0,1


In [ ]:
processed_dataIps.head(3)

NaN,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,674.0,675.0,676.0,677.0,678.0,679.0,680.0,681.0,categorie,id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.68,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.12,0.0,0.0,0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.46,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.12,0.0,0.0,0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.68,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.12,0.0,0.0,0,1


In [43]:
def convert_object_to_numeric(data):
  for col in data.columns:
      if data[col].dtype == 'object':
          data[col] = pd.to_numeric(data[col], errors='coerce')
  return data

In [ ]:
"""print("imu procces types : ", processed_dataImu.dtypes)
print("emg procces types : ", processed_dataEmg.dtypes)
print("ips procces types : ", processed_dataIps.dtypes)"""

'print("imu procces types : ", processed_dataImu.dtypes)\nprint("emg procces types : ", processed_dataEmg.dtypes)\nprint("ips procces types : ", processed_dataIps.dtypes)'

In [44]:
numeric_dataImu = convert_object_to_numeric(processed_dataImu)
numeric_dataEmg = convert_object_to_numeric(processed_dataEmg)
numeric_dataIps = convert_object_to_numeric(processed_dataIps)

In [45]:
numeric_dataImu.head(3)

Unnamed: 0,Head_Acc_X,Head_Acc_Y,Head_Acc_Z,Head_Gyr_X,Head_Gyr_Y,Head_Gyr_Z,Head_Roll,Head_Pitch,Head_Yaw,Waist_Acc_X,...,R_F_Acc_Y,R_F_Acc_Z,R_F_Gyr_X,R_F_Gyr_Y,R_F_Gyr_Z,R_F_Roll,R_F_Pitch,R_F_Yaw,categorie,id
0,8.902975,0.412529,4.052022,-0.015015,0.010260,-0.009929,-12.639535,-10.234849,30.077293,9.578385,...,-1.903201,9.019931,-0.027493,0.009596,-0.008538,10.781965,35.531989,-17.776944,0,1
1,8.912331,0.391603,4.045956,-0.010874,0.014445,-0.006227,-12.643968,-10.226859,30.069993,9.540294,...,-1.942165,9.052294,-0.016795,0.007321,-0.009379,10.773012,35.535744,-17.793866,0,1
2,8.889990,0.384217,4.039017,-0.013686,0.012110,-0.005442,-12.647921,-10.220573,30.063655,9.561864,...,-1.965231,9.028006,-0.009752,0.004455,-0.011313,10.768550,35.536932,-17.806589,0,1


In [ ]:
print("imu procces types : ", numeric_dataImu.dtypes)
print("emg procces types : ", numeric_dataEmg.dtypes)
print("ips procces types : ", numeric_dataIps.dtypes)

imu procces types :  Unnamed: 0
Head_Acc_X     float64
Head_Acc_Y     float64
Head_Acc_Z     float64
Head_Gyr_X     float64
Head_Gyr_Y     float64
Head_Gyr_Z     float64
Head_Roll      float64
Head_Pitch     float64
Head_Yaw       float64
Waist_Acc_X    float64
Waist_Acc_Y    float64
Waist_Acc_Z    float64
Waist_Gyr_X    float64
Waist_Gyr_Y    float64
Waist_Gyr_Z    float64
Waist_Roll     float64
Waist_Pitch    float64
Waist_Yaw      float64
L_H_Acc_X      float64
L_H_Acc_Y      float64
L_H_Acc_Z      float64
L_H_Gyr_X      float64
L_H_Gyr_Y      float64
L_H_Gyr_Z      float64
L_H_Roll       float64
L_H_Pitch      float64
L_H_Yaw        float64
R_H_Acc_X      float64
R_H_Acc_Y      float64
R_H_Acc_Z      float64
R_H_Gyr_X      float64
R_H_Gyr_Y      float64
R_H_Gyr_Z      float64
R_H_Roll       float64
R_H_Pitch      float64
R_H_Yaw        float64
L_F_Acc_X      float64
L_F_Acc_Y      float64
L_F_Acc_Z      float64
L_F_Gyr_X      float64
L_F_Gyr_Y      float64
L_F_Gyr_Z      float64
L_

In [ ]:
# une etape important pour la memoire !!!
# Les colonnes de type float64 et int64 occupent beaucoup de mémoire. convertir en types plus compacts (float32 ou int32)
def reduce_memory_usage(df):
  for col in df.columns:
      col_type = df[col].dtype
      if col_type == 'float64':
          df[col] = df[col].astype('float32')
      elif col_type == 'int64':
          df[col] = df[col].astype('int32')
  return df

In [ ]:
processed_dataImu = reduce_memory_usage(numeric_dataImu)
processed_dataEmg = reduce_memory_usage(numeric_dataEmg)
processed_dataIps = reduce_memory_usage(numeric_dataIps)

In [ ]:
processed_dataImu.head(3)

Unnamed: 0,Head_Acc_X,Head_Acc_Y,Head_Acc_Z,Head_Gyr_X,Head_Gyr_Y,Head_Gyr_Z,Head_Roll,Head_Pitch,Head_Yaw,Waist_Acc_X,...,R_F_Acc_Y,R_F_Acc_Z,R_F_Gyr_X,R_F_Gyr_Y,R_F_Gyr_Z,R_F_Roll,R_F_Pitch,R_F_Yaw,categorie,id
0,8.902975,0.412529,4.052022,-0.015015,0.010260,-0.009929,-12.639535,-10.234849,30.077293,9.578385,...,-1.903201,9.019931,-0.027493,0.009596,-0.008538,10.781965,35.531990,-17.776943,0,1
1,8.912331,0.391603,4.045956,-0.010874,0.014445,-0.006227,-12.643968,-10.226859,30.069992,9.540294,...,-1.942165,9.052294,-0.016795,0.007321,-0.009379,10.773012,35.535744,-17.793865,0,1
2,8.889990,0.384217,4.039017,-0.013686,0.012110,-0.005442,-12.647921,-10.220573,30.063656,9.561864,...,-1.965231,9.028006,-0.009752,0.004455,-0.011313,10.768550,35.536934,-17.806589,0,1


In [ ]:
print("imu procces types : ", processed_dataImu.dtypes)
print("emg procces types : ", processed_dataEmg.dtypes)
print("ips procces types : ", processed_dataIps.dtypes)

imu procces types :  Unnamed: 0
Head_Acc_X     float32
Head_Acc_Y     float32
Head_Acc_Z     float32
Head_Gyr_X     float32
Head_Gyr_Y     float32
Head_Gyr_Z     float32
Head_Roll      float32
Head_Pitch     float32
Head_Yaw       float32
Waist_Acc_X    float32
Waist_Acc_Y    float32
Waist_Acc_Z    float32
Waist_Gyr_X    float32
Waist_Gyr_Y    float32
Waist_Gyr_Z    float32
Waist_Roll     float32
Waist_Pitch    float32
Waist_Yaw      float32
L_H_Acc_X      float32
L_H_Acc_Y      float32
L_H_Acc_Z      float32
L_H_Gyr_X      float32
L_H_Gyr_Y      float32
L_H_Gyr_Z      float32
L_H_Roll       float32
L_H_Pitch      float32
L_H_Yaw        float32
R_H_Acc_X      float32
R_H_Acc_Y      float32
R_H_Acc_Z      float32
R_H_Gyr_X      float32
R_H_Gyr_Y      float32
R_H_Gyr_Z      float32
R_H_Roll       float32
R_H_Pitch      float32
R_H_Yaw        float32
L_F_Acc_X      float32
L_F_Acc_Y      float32
L_F_Acc_Z      float32
L_F_Gyr_X      float32
L_F_Gyr_Y      float32
L_F_Gyr_Z      float32
L_

In [46]:
def clean_missing_values(processed_data, threshold=0.5):

  zero_threshold = (processed_data == 0).mean(axis=1)  # Calcul du pourcentage de zéros par ligne
  rows_to_delete = zero_threshold[zero_threshold >= threshold]
  #print(f"Lignes supprimées (trop de zéros):\n{rows_to_delete}")
  processed_data = processed_data.loc[zero_threshold < threshold]
  print(f"Nombre de lignes avant suppression: {len(zero_threshold)}")
  print(f"Nombre de lignes après suppression: {len(processed_data)}")
  numeric_cols = processed_data.select_dtypes(include=[np.number])
  non_numeric_cols = processed_data.select_dtypes(exclude=[np.number])
  missing_percentage = numeric_cols.isna().mean()
  numeric_cols_cleaned = numeric_cols.loc[:, missing_percentage < threshold]
  numeric_cols_cleaned = numeric_cols_cleaned.apply(lambda col: col.fillna(col.median()), axis=0)
  processed_data_cleaned = pd.concat([non_numeric_cols, numeric_cols_cleaned], axis=1)

  return processed_data_cleaned

In [47]:
#cleaned_data = clean_missing_values(processed_data, porcentage 0.5)
cleaned_dataImu = clean_missing_values(processed_dataImu, threshold=0.5)
cleaned_dataEmg = clean_missing_values(processed_dataEmg, threshold=0.5)
cleaned_dataIps = clean_missing_values(processed_dataIps, threshold=0.5)

Nombre de lignes avant suppression: 551504
Nombre de lignes après suppression: 548712
Nombre de lignes avant suppression: 11029891
Nombre de lignes après suppression: 11029891
Nombre de lignes avant suppression: 343310
Nombre de lignes après suppression: 171743


In [48]:
cleaned_dataEmg.head(3)

Unnamed: 0,R_Vlat,R_RF,R_ST,R_TA,L_Vlat,L_RF,L_ST,L_TA,R_MG,R_LG,R_SOL,R_IL,L_MG,L_LG,L_SOL,L_IL,categorie,id
0,-0.704956,1.309204,-2.316284,-20.242310,-6.747437,2.920532,-1.208496,-6.948853,-0.906372,-12.387085,54.180908,-2.5177,-0.704956,10.876465,-3.52478,-3.52478,0,1
1,2.014160,1.309204,-2.316284,-16.415405,-1.208496,2.618408,-1.208496,-6.948853,-0.906372,-12.387085,54.180908,-2.5177,-0.704956,10.876465,-3.52478,-3.52478,0,1
2,-0.704956,1.309204,-2.316284,-16.314697,-1.107788,1.107788,-1.208496,-6.948853,-0.906372,-12.387085,54.180908,-2.5177,-0.704956,10.876465,-3.52478,-3.52478,0,1


In [ ]:
cleaned_dataImu.head(3)

Unnamed: 0,Head_Acc_X,Head_Acc_Y,Head_Acc_Z,Head_Gyr_X,Head_Gyr_Y,Head_Gyr_Z,Head_Roll,Head_Pitch,Head_Yaw,Waist_Acc_X,...,R_F_Acc_Y,R_F_Acc_Z,R_F_Gyr_X,R_F_Gyr_Y,R_F_Gyr_Z,R_F_Roll,R_F_Pitch,R_F_Yaw,categorie,id
0,8.902975,0.412529,4.052022,-0.015015,0.010260,-0.009929,-12.639535,-10.234849,30.077293,9.578385,...,-1.903201,9.019931,-0.027493,0.009596,-0.008538,10.781965,35.531990,-17.776943,0,1
1,8.912331,0.391603,4.045956,-0.010874,0.014445,-0.006227,-12.643968,-10.226859,30.069992,9.540294,...,-1.942165,9.052294,-0.016795,0.007321,-0.009379,10.773012,35.535744,-17.793865,0,1
2,8.889990,0.384217,4.039017,-0.013686,0.012110,-0.005442,-12.647921,-10.220573,30.063656,9.561864,...,-1.965231,9.028006,-0.009752,0.004455,-0.011313,10.768550,35.536934,-17.806589,0,1


In [49]:
print("Catégories dans IPS:", cleaned_dataIps['categorie'].unique())
print("Catégories dans IMU:", cleaned_dataImu['categorie'].unique())
print("Catégories dans EMG:", cleaned_dataEmg['categorie'].unique())

Catégories dans IPS: [0 1 2 3]
Catégories dans IMU: [0 1 2 3]
Catégories dans EMG: [0 1 2 3]


In [ ]:
# juste for test
"""def estimate_frequency(sample_count, duration_seconds):

    freq_hz = sample_count / duration_seconds
    return freq_hz
emg_sample_count = cleaned_dataEmg.shape[0]
imu_sample_count = cleaned_dataImu.shape[0]
ips_sample_count = cleaned_dataIps.shape[0]

emg_freq = estimate_frequency(emg_sample_count, 2000)
imu_freq = estimate_frequency(imu_sample_count, 100)
ips_freq = estimate_frequency(ips_sample_count, 35)

print(f"EMG Frequency: {emg_freq:.2f} Hz")
print(f"IMU Frequency: {imu_freq:.2f} Hz")
print(f"IPS Frequency: {ips_freq:.2f} Hz")"""


'def estimate_frequency(sample_count, duration_seconds):\n\n    freq_hz = sample_count / duration_seconds\n    return freq_hz\nemg_sample_count = cleaned_dataEmg.shape[0]\nimu_sample_count = cleaned_dataImu.shape[0]\nips_sample_count = cleaned_dataIps.shape[0]\n\nemg_freq = estimate_frequency(emg_sample_count, 2000)\nimu_freq = estimate_frequency(imu_sample_count, 100)\nips_freq = estimate_frequency(ips_sample_count, 35)\n\nprint(f"EMG Frequency: {emg_freq:.2f} Hz")\nprint(f"IMU Frequency: {imu_freq:.2f} Hz")\nprint(f"IPS Frequency: {ips_freq:.2f} Hz")'

In [ ]:
"""def plot_data_comparison(original_df, resampled_df, modality_name):

  plt.figure(figsize=(14, 6))
  # Comparaison de la première colonne
  original_column = original_df.columns[0]
  resampled_column = resampled_df.columns[0]
  # Plot des données originales
  plt.subplot(1, 2, 1)
  plt.plot(original_df.index, original_df[original_column], label='Original')
  plt.title(f'{modality_name} - Original Data')
  plt.xlabel('Time Index')
  plt.ylabel('Value')
  plt.legend()
  # Plot des données rééchantillonnées
  plt.subplot(1, 2, 2)
  plt.plot(resampled_df.index, resampled_df[resampled_column], label='Resampled', color='orange')
  plt.title(f'{modality_name} - Resampled Data')
  plt.xlabel('Time Index')
  plt.ylabel('Value')
  plt.legend()

  plt.tight_layout()
  plt.show()"""

"def plot_data_comparison(original_df, resampled_df, modality_name):\n\n  plt.figure(figsize=(14, 6))\n  # Comparaison de la première colonne\n  original_column = original_df.columns[0]\n  resampled_column = resampled_df.columns[0]\n  # Plot des données originales\n  plt.subplot(1, 2, 1)\n  plt.plot(original_df.index, original_df[original_column], label='Original')\n  plt.title(f'{modality_name} - Original Data')\n  plt.xlabel('Time Index')\n  plt.ylabel('Value')\n  plt.legend()\n  # Plot des données rééchantillonnées\n  plt.subplot(1, 2, 2)\n  plt.plot(resampled_df.index, resampled_df[resampled_column], label='Resampled', color='orange')\n  plt.title(f'{modality_name} - Resampled Data')\n  plt.xlabel('Time Index')\n  plt.ylabel('Value')\n  plt.legend()\n\n  plt.tight_layout()\n  plt.show()"

In [50]:
def resample_data(df, original_freq, target_freq, ignore_columns=['categorie', 'id'], debug=False):
  if original_freq <= 0 or target_freq <= 0:
    raise ValueError("Les fréquences d'échantillonnage doivent être positives et non nulles.")
  if original_freq == target_freq:
    return df
  if debug:
    print(f"Rééchantillonnage de {original_freq} Hz à {target_freq} Hz.")

  ignore_columns = [col for col in ignore_columns if col in df.columns]
  columns_to_resample = df.drop(columns=ignore_columns)

  if 'Unnamed: 0' in columns_to_resample.columns:
    columns_to_resample = columns_to_resample.drop(columns=['Unnamed: 0'])

  df['time'] = pd.date_range(
    start='00:00:00',
    periods=columns_to_resample.shape[0],
    freq=f'{int(1e6 / original_freq)}us'
  )
  df.set_index('time', inplace=True)

  resampled_df = columns_to_resample.resample(f'{int(1e6 / target_freq)}us').interpolate(method='linear')

  for col in ignore_columns:
    if col in df.columns:
      try:
        resampled_df[col] = df[col].resample(f'{int(1e6 / target_freq)}us').ffill()
        resampled_df[col] = resampled_df[col].astype(df[col].dtype)
      except Exception as e:
        print(f"Erreur pendant le rééchantillonnage de la colonne ignorée '{col}' :", e)
        raise

  resampled_df.reset_index(inplace=True)
  resampled_df.drop(columns=['time'], inplace=True, errors='ignore')

  if debug:
    print(f"Nombre de points après interpolation : {resampled_df.shape[0]}")

  return resampled_df

In [ ]:
"""print(cleaned_dataEmg.head(3))
print(f"Type d'index initial : {type(cleaned_dataEmg.index)}")"""

'print(cleaned_dataEmg.head(3))\nprint(f"Type d\'index initial : {type(cleaned_dataEmg.index)}")'

In [53]:
ips_resampled = resample_data(cleaned_dataIps, original_freq=60, target_freq=60)
emg_resampled = resample_data(cleaned_dataEmg, original_freq=2000, target_freq=60)
imu_resampled = resample_data(cleaned_dataImu, original_freq=100, target_freq=60)

In [54]:
imu_resampled.head(3)

Unnamed: 0,Head_Acc_X,Head_Acc_Y,Head_Acc_Z,Head_Gyr_X,Head_Gyr_Y,Head_Gyr_Z,Head_Roll,Head_Pitch,Head_Yaw,Waist_Acc_X,...,R_F_Acc_Y,R_F_Acc_Z,R_F_Gyr_X,R_F_Gyr_Y,R_F_Gyr_Z,R_F_Roll,R_F_Pitch,R_F_Yaw,categorie,id
0,8.902975,0.412529,4.052022,-0.015015,0.010260,-0.009929,-12.639535,-10.234849,30.077293,9.578385,...,-1.903201,9.019931,-0.027493,0.009596,-0.008538,10.781965,35.531989,-17.776944,0,1
1,8.902771,0.412443,4.052364,-0.015010,0.010255,-0.009926,-12.641062,-10.232953,30.077219,9.578046,...,-1.903192,9.019924,-0.027488,0.009595,-0.008536,10.782005,35.531819,-17.774744,0,1
2,8.902568,0.412356,4.052706,-0.015005,0.010250,-0.009923,-12.642589,-10.231057,30.077146,9.577707,...,-1.903184,9.019917,-0.027483,0.009593,-0.008533,10.782044,35.531648,-17.772543,0,1


In [55]:
emg_resampled.head(3)

Unnamed: 0,R_Vlat,R_RF,R_ST,R_TA,L_Vlat,L_RF,L_ST,L_TA,R_MG,R_LG,R_SOL,R_IL,L_MG,L_LG,L_SOL,L_IL,categorie,id
0,-0.704956,1.309204,-2.316284,-20.242310,-6.747437,2.920532,-1.208496,-6.948853,-0.906372,-12.387085,54.180908,-2.517700,-0.704956,10.876465,-3.524780,-3.524780,0,1
1,-0.719861,1.338611,-2.321118,-20.124683,-6.777246,2.865344,-1.194397,-7.264673,-0.937793,-12.321021,53.984729,-2.537842,-0.694482,10.852698,-3.522766,-3.511084,0,1
2,-0.734766,1.368018,-2.325952,-20.007056,-6.807056,2.810156,-1.180298,-7.580493,-0.969214,-12.254956,53.788550,-2.557983,-0.684009,10.828931,-3.520752,-3.497388,0,1


In [56]:
ips_resampled.head(3)

NaN,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,674.0,675.0,676.0,677.0,678.0,679.0,680.0,681.0,categorie,id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.68,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.12,0.0,0.0,0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.46,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.12,0.0,0.0,0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.68,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.12,0.0,0.0,0,1


In [57]:
#cleaned_data.shape
print("cleaned_cleaned Imu : ", cleaned_dataImu.shape)
print("cleaned_cleaned Emg : ", cleaned_dataEmg.shape)
print("cleaned_cleaned ips : ", cleaned_dataIps.shape)

cleaned_cleaned Imu :  (548712, 56)
cleaned_cleaned Emg :  (11029891, 18)
cleaned_cleaned ips :  (171743, 684)


In [58]:
print("rééchantillonné Imu : ", imu_resampled.shape)
print("rééchantillonné Emg : ", emg_resampled.shape)
print("rééchantillonné ips : ", ips_resampled.shape)

rééchantillonné Imu :  (329240, 56)
rééchantillonné Emg :  (330910, 18)
rééchantillonné ips :  (171743, 684)


In [59]:
print("Catégories dans IPS:", ips_resampled['categorie'].unique())
print("Catégories dans IMU:", imu_resampled['categorie'].unique())
print("Catégories dans EMG:", emg_resampled['categorie'].unique())

Catégories dans IPS: [0 1 2 3]
Catégories dans IMU: [0 1 2 3]
Catégories dans EMG: [0 1 2 3]


In [ ]:
"""plot_data_comparison(cleaned_dataEmg, emg_resampled, 'EMG')
plot_data_comparison(cleaned_dataImu, imu_resampled, 'IMU')
plot_data_comparison(cleaned_dataIps, ips_resampled, 'IPS')"""

"plot_data_comparison(cleaned_dataEmg, emg_resampled, 'EMG')\nplot_data_comparison(cleaned_dataImu, imu_resampled, 'IMU')\nplot_data_comparison(cleaned_dataIps, ips_resampled, 'IPS')"

In [62]:
def normalize_data(data, ignore_columns=['id', 'categorie']):
  # Vérification des colonnes à ignorer
  ignore_columns = [col for col in ignore_columns if col in data.columns]
  if len(ignore_columns) < len(set(ignore_columns)):
    print("alert : certaines colonnes dans 'ignore_columns' n'existent pas dans le DataFrame.")
  # Séparer les colonnes à ignorer (id, categorie, etc.)
  to_ignore = data[ignore_columns]
  to_normalize = data.drop(columns=ignore_columns)
  numeric_cols = to_normalize.select_dtypes(include=[float, int]).drop(columns=['categorie'], errors='ignore')
  if 'categorie' in numeric_cols.columns:
    numeric_cols['categorie'] = numeric_cols['categorie'].astype('object')
  # Normalisation des colonnes numériques
  scaler = StandardScaler()
  normalized_numeric = pd.DataFrame(
      scaler.fit_transform(numeric_cols),
      columns=numeric_cols.columns,
      index=numeric_cols.index
  )
  # Réintégration des colonnes non numériques ignorées
  non_numeric_cols = to_normalize.select_dtypes(exclude=[float, int])
  if not non_numeric_cols.empty:
    print("Notif : Les colonnes non numériques ont été exclues de la normalisation.")
  # Reconstruction du DataFrame final
  normalized_data = pd.concat([normalized_numeric.reset_index(drop=True), non_numeric_cols.reset_index(drop=True), to_ignore.reset_index(drop=True)], axis=1)
  # Restaurer l'ordre original des colonnes
  normalized_data = normalized_data[data.columns]

  return normalized_data

In [63]:
normalized_dataImu = normalize_data(imu_resampled)
normalized_dataEmg = normalize_data(emg_resampled)
normalized_dataIps = normalize_data(ips_resampled)

In [ ]:
normalized_dataImu.head(3)

Unnamed: 0,Head_Acc_X,Head_Acc_Y,Head_Acc_Z,Head_Gyr_X,Head_Gyr_Y,Head_Gyr_Z,Head_Roll,Head_Pitch,Head_Yaw,Waist_Acc_X,...,R_F_Acc_Y,R_F_Acc_Z,R_F_Gyr_X,R_F_Gyr_Y,R_F_Gyr_Z,R_F_Roll,R_F_Pitch,R_F_Yaw,categorie,id
0,0.246282,1.149197,0.577646,-0.425563,-0.293250,-0.360639,0.263954,0.299865,0.836070,0.908151,...,-1.060534,1.510291,-3.192521,1.195564,-1.348484,1.392718,1.488743,0.377717,0,1
1,0.246123,1.149059,0.577746,-0.425465,-0.293279,-0.360612,0.263918,0.299928,0.836069,0.907332,...,-1.060527,1.510273,-3.191936,1.195347,-1.348171,1.392724,1.488734,0.377755,0,1
2,0.245963,1.148922,0.577845,-0.425367,-0.293308,-0.360584,0.263883,0.299990,0.836067,0.906515,...,-1.060519,1.510253,-3.191351,1.195129,-1.347858,1.392730,1.488726,0.377793,0,1


In [ ]:
normalized_dataEmg.head(3)

Unnamed: 0,R_Vlat,R_RF,R_ST,R_TA,L_Vlat,L_RF,L_ST,L_TA,R_MG,R_LG,R_SOL,R_IL,L_MG,L_LG,L_SOL,L_IL,categorie,id
0,0.040085,0.086058,0.086466,-0.414007,-0.138624,0.119031,0.109122,-0.073721,0.081198,-1.547513,2.521937,-0.187355,0.045367,0.578955,-0.360731,0.364890,0,1
1,0.039749,0.087394,0.086233,-0.410660,-0.139202,0.116897,0.109923,-0.079790,0.080053,-1.539252,2.512969,-0.189012,0.045670,0.577939,-0.360563,0.364903,0,1
2,0.039412,0.088730,0.085999,-0.407312,-0.139780,0.114764,0.110724,-0.085860,0.078908,-1.530991,2.504000,-0.190669,0.045973,0.576922,-0.360394,0.364916,0,1


In [ ]:
normalized_dataIps.head(3)

NaN,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,674.0,675.0,676.0,677.0,678.0,679.0,680.0,681.0,categorie,id
0,0.0,0.0,0.0,0.0,0.0,-0.314095,-0.279942,0.605183,-0.444921,0.0,...,0.0,0.0,0.0,-0.465096,-0.348381,-0.342469,0.0,0.0,0,1
1,0.0,0.0,0.0,0.0,0.0,-0.314095,-0.279942,0.477971,-0.444921,0.0,...,0.0,0.0,0.0,-0.465096,-0.348381,-0.342469,0.0,0.0,0,1
2,0.0,0.0,0.0,0.0,0.0,-0.314095,-0.279942,0.605183,-0.444921,0.0,...,0.0,0.0,0.0,-0.465096,-0.348381,-0.342469,0.0,0.0,0,1


sauvgarder les modalities !!

In [64]:
processed_data_path = '/content/drive/MyDrive/MovePort/processedData'
if not os.path.exists(processed_data_path): os.makedirs(processed_data_path)

In [65]:
emg_normalized = pd.DataFrame(normalized_dataEmg)
imu_normalized = pd.DataFrame(normalized_dataImu)
ips_normalized = pd.DataFrame(normalized_dataIps)

In [66]:
emg_normalized.to_csv(os.path.join(processed_data_path, 'emg_normalized.csv'), index=False)
imu_normalized.to_csv(os.path.join(processed_data_path, 'imu_normalized.csv'), index=False)
ips_normalized.to_csv(os.path.join(processed_data_path, 'ips_normalized.csv'), index=False)

In [67]:
emg_normalized = pd.read_csv(f"{processed_data_path}/emg_normalized.csv")
imu_normalized = pd.read_csv(f"{processed_data_path}/imu_normalized.csv")
ips_normalized = pd.read_csv(f"{processed_data_path}/ips_normalized.csv")

In [ ]:
#imu_normalized.value_counts()

In [ ]:
emg_normalized.head(3)

,R_Vlat,R_RF,R_ST,R_TA,L_Vlat,L_RF,L_ST,L_TA,R_MG,R_LG,R_SOL,R_IL,L_MG,L_LG,L_SOL,L_IL,categorie,id
0,0.040085,0.086058,0.086466,-0.414007,-0.138624,0.119031,0.109122,-0.073721,0.081198,-1.547513,2.521937,-0.187355,0.045367,0.578955,-0.360731,0.364890,0,1
1,0.039749,0.087394,0.086233,-0.410660,-0.139202,0.116897,0.109923,-0.079790,0.080053,-1.539252,2.512969,-0.189012,0.045670,0.577939,-0.360563,0.364903,0,1
2,0.039412,0.088730,0.085999,-0.407312,-0.139780,0.114764,0.110724,-0.085860,0.078908,-1.530991,2.504000,-0.190669,0.045973,0.576922,-0.360394,0.364916,0,1


In [68]:
print("Valeurs uniques dans EMG avant conversion :", emg_normalized['categorie'].unique())
print("Valeurs uniques dans EMG après conversion :", emg_normalized['categorie'].astype(int).unique())

Valeurs uniques dans EMG avant conversion : [0 1 2 3]
Valeurs uniques dans EMG après conversion : [0 1 2 3]


In [69]:
print("Distribution des catégories EMG avant conversion :")
print(emg_normalized['categorie'].value_counts())
print("Distribution des catégories EMG après conversion :")
print(emg_normalized['categorie'].astype(int).value_counts())


Distribution des catégories EMG avant conversion :
categorie
0    88874
3    84129
2    82874
1    75033
Name: count, dtype: int64
Distribution des catégories EMG après conversion :
categorie
0    88874
3    84129
2    82874
1    75033
Name: count, dtype: int64


In [ ]:
"""print("Lignes avec categorie = 2 dans EMG :")
print(emg_normalized[emg_normalized['categorie'] == 2])
print("Lignes avec categorie = 3 dans EMG :")
print(emg_normalized[emg_normalized['categorie'] == 3])"""

'print("Lignes avec categorie = 2 dans EMG :")\nprint(emg_normalized[emg_normalized[\'categorie\'] == 2])\nprint("Lignes avec categorie = 3 dans EMG :")\nprint(emg_normalized[emg_normalized[\'categorie\'] == 3])'

In [70]:
print("Catégories dans IPS:", ips_normalized['categorie'].unique())
print("Catégories dans IMU:", imu_normalized['categorie'].unique())
print("Catégories dans EMG:", emg_normalized['categorie'].unique())

Catégories dans IPS: [0 1 2 3]
Catégories dans IMU: [0 1 2 3]
Catégories dans EMG: [0 1 2 3]


In [71]:
print("Type de 'categorie' dans IPS:", ips_normalized['categorie'].dtype)
print("Type de 'categorie' dans IMU:", imu_normalized['categorie'].dtype)
print("Type de 'categorie' dans EMG:", emg_normalized['categorie'].dtype)

Type de 'categorie' dans IPS: int64
Type de 'categorie' dans IMU: int64
Type de 'categorie' dans EMG: int64


In [ ]:
"""# Corriger le type de 'categorie' après le rééchantillonnage
ips_normalized['categorie'] = ips_normalized['categorie'].astype(int)
imu_normalized['categorie'] = imu_normalized['categorie'].astype(int)
emg_normalized['categorie'] = emg_normalized['categorie'].astype(int)"""

"# Corriger le type de 'categorie' après le rééchantillonnage\nips_normalized['categorie'] = ips_normalized['categorie'].astype(int)\nimu_normalized['categorie'] = imu_normalized['categorie'].astype(int)\nemg_normalized['categorie'] = emg_normalized['categorie'].astype(int)"

In [72]:
# Compter les catégories dans chaque modalité avant normalisation ou autre traitement
print("Avant traitement :")
print("IPS Categories Count:", ips_normalized['categorie'].value_counts())
print("IMU Categories Count:", imu_normalized['categorie'].value_counts())
print("EMG Categories Count:", emg_normalized['categorie'].value_counts())

Avant traitement :
IPS Categories Count: categorie
3    67075
2    46403
1    45407
0    12858
Name: count, dtype: int64
IMU Categories Count: categorie
0    88450
3    84117
2    81650
1    75023
Name: count, dtype: int64
EMG Categories Count: categorie
0    88874
3    84129
2    82874
1    75033
Name: count, dtype: int64


In [73]:
def calculate_category_distribution(modality_data, target_column='categorie'):
  return modality_data[target_column].value_counts()

ips_distribution = calculate_category_distribution(ips_normalized, target_column='categorie')
imu_distribution = calculate_category_distribution(imu_normalized, target_column='categorie')
emg_distribution = calculate_category_distribution(emg_normalized, target_column='categorie')

distribution_table = pd.DataFrame({
  'ips': ips_distribution,
  'imu': imu_distribution,
  'emg': emg_distribution
}).fillna(0).astype(int)

print(distribution_table)


             ips    imu    emg
categorie                     
0          12858  88450  88874
1          45407  75023  75033
2          46403  81650  82874
3          67075  84117  84129


In [74]:
numbreMean = pd.DataFrame(distribution_table, index=[0, 1, 2, 3])
numbreMean

,ips,imu,emg
0,12858,88450,88874
1,45407,75023,75033
2,46403,81650,82874
3,67075,84117,84129


In [75]:
moyenne_min = numbreMean.mean().min()
moyenne_min

42935.75

In [76]:
valeur_arrondie = int(round(moyenne_min, -3))  # Arrondir à la centaine la plus proche
print("Valeur arrondie la plus proche:", valeur_arrondie)

Valeur arrondie la plus proche: 43000


In [77]:
from sklearn.utils import resample
def align_modalities_to_target(data, target_column, target_size, random_state=42):

  aligned_data = []
  # Parcourir chaque catégorie
  for category in data[target_column].unique():
    category_data = data[data[target_column] == category]
    current_size = len(category_data)

    if current_size > target_size:
      # Réduire si la catégorie a plus de données que la cible
      reduced_data = category_data.sample(n=target_size, random_state=random_state)
      aligned_data.append(reduced_data)
    elif current_size < target_size:
      # Augmenter si la catégorie a moins de données que la cible
      increased_data = resample(
        category_data,
        replace=True,  # Over-sampling
        n_samples=target_size,
        random_state=random_state
      )
      aligned_data.append(increased_data)
    else:
      # Si déjà aligné, conserver tel quel
      aligned_data.append(category_data)

  return pd.concat(aligned_data).reset_index(drop=True)

In [78]:
print("normilized Imu : ", imu_normalized.shape)
print("normilized Emg : ", emg_normalized.shape)
print("normilized ips : ", ips_normalized.shape)

normilized Imu :  (329240, 56)
normilized Emg :  (330910, 18)
normilized ips :  (171743, 684)


In [79]:
ips_aligned = align_modalities_to_target(ips_normalized, target_column='categorie', target_size=valeur_arrondie)
imu_aligned = align_modalities_to_target(imu_normalized, target_column='categorie', target_size=valeur_arrondie)
emg_aligned = align_modalities_to_target(emg_normalized, target_column='categorie', target_size=valeur_arrondie)

In [80]:
print("aligned Imu : ", imu_aligned.shape)
print("aligned Emg : ", emg_aligned.shape)
print("aligned ips : ", ips_aligned.shape)

aligned Imu :  (172000, 56)
aligned Emg :  (172000, 18)
aligned ips :  (172000, 684)


In [81]:
ips_distribution = calculate_category_distribution(ips_aligned, target_column='categorie')
imu_distribution = calculate_category_distribution(imu_aligned, target_column='categorie')
emg_distribution = calculate_category_distribution(emg_aligned, target_column='categorie')

distribution_table = pd.DataFrame({
    'ips': ips_distribution,
    'imu': imu_distribution,
    'emg': emg_distribution
}).fillna(0).astype(int)

print(distribution_table)

             ips    imu    emg
categorie                     
0          43000  43000  43000
1          43000  43000  43000
2          43000  43000  43000
3          43000  43000  43000


In [83]:
# 'id' et 'categorie' à part
categories = emg_aligned[['id', 'categorie']]
# Supprimer les colonnes redondantes des DataFrames individuels
emg_data = emg_aligned.drop(columns=['id', 'categorie'])
ips_data = ips_aligned.drop(columns=['id', 'categorie'])
imu_data = imu_aligned.drop(columns=['id', 'categorie'])
# Concaténer les données sans les colonnes dupliquées
all_aligned_data = pd.concat([emg_data, ips_data, imu_data, categories], axis=1)

In [84]:
# Vérifier la forme du DataFrame résultant
print("Shape de all_aligned_data :", all_aligned_data.shape)

# Vérifier les premières lignes pour s'assurer que les données sont correctes
print(all_aligned_data.head())

Shape de all_aligned_data : (172000, 754)
     R_Vlat      R_RF      R_ST      R_TA    L_Vlat      L_RF      L_ST  \
0 -1.255556  0.646434 -0.536403  0.857758  0.216019 -0.022277  0.104132   
1  0.494693 -0.159464 -0.085246 -1.986042 -0.941090 -1.427089 -0.549837   
2 -0.758321 -0.656587 -0.295860 -5.564186  1.393137 -2.057526  0.131601   
3 -0.985203  0.227583 -0.996253 -0.136097 -0.523258  0.491972  0.070631   
4  0.144257  0.588693  0.085851 -3.120032  0.359972 -0.046455  0.121175   

       L_TA      R_MG      R_LG  ...  R_F_Acc_Y  R_F_Acc_Z  R_F_Gyr_X  \
0  0.081469 -0.036459  0.368538  ...   0.307654   0.099212  -1.213839   
1  2.560845 -0.069939 -0.533157  ...  -1.252182   0.441247   0.810265   
2 -1.027288  0.189120  0.029485  ...   0.130687   0.366296   0.129835   
3 -1.310905  0.123376 -0.168818  ...  -0.852217   0.573079   0.239303   
4  4.379372 -0.421386 -1.026465  ...  -1.178759   0.441207   0.173372   

   R_F_Gyr_Y  R_F_Gyr_Z  R_F_Roll  R_F_Pitch   R_F_Yaw  id  categori

In [82]:
"""# Concaténation des modalités alignées
all_aligned_data = pd.concat([emg_aligned, ips_aligned ,imu_aligned], axis=0)
# Vérification des distributions
"""

'# Concaténation des modalités alignées\nall_aligned_data = pd.concat([emg_aligned, ips_aligned ,imu_aligned], axis=0)\n# Vérification des distributions\n'

In [85]:
print("Final Category Distribution Across Modalities:")
print(all_aligned_data['categorie'].value_counts())

Final Category Distribution Across Modalities:
categorie
0    43000
1    43000
2    43000
3    43000
Name: count, dtype: int64


In [86]:
emg_aligned.to_csv(os.path.join(processed_data_path, 'emg_aligned.csv'), index=False)
imu_aligned.to_csv(os.path.join(processed_data_path, 'imu_aligned.csv'), index=False)
ips_aligned.to_csv(os.path.join(processed_data_path, 'ips_aligned.csv'), index=False)

In [87]:
all_aligned_data.to_csv(os.path.join(processed_data_path, 'all_aligned_data.csv'), index=False)

In [4]:
processed_data_path = '/content/drive/MyDrive/MovePort/processedData'

emg_aligned = pd.read_csv(f"{processed_data_path}/emg_aligned.csv")
imu_aligned = pd.read_csv(f"{processed_data_path}/imu_aligned.csv")
ips_aligned = pd.read_csv(f"{processed_data_path}/ips_aligned.csv")

In [5]:
all_aligned_data = pd.read_csv(f"{processed_data_path}/all_aligned_data.csv")

In [6]:
# total colonnes doit être 752 + cat et id = 754
all_aligned_data.shape

(172000, 754)

In [7]:
"""# set id and categorie to end of columns (comme ça on sait qu'ils sont en fin)
def move_columns_to_end(df, columns_to_move):
  remaining_columns = [col for col in df.columns if col not in columns_to_move]
  return df[remaining_columns + columns_to_move]"""

In [ ]:
#print("Colonnes dans les données fusionnées :", all_aligned_data.columns == 'categorie')

In [ ]:
"""# Réorganiser les colonnes pour s'assurer que 'categorie' et 'id' sont à la fin
cols = [col for col in all_aligned_data.columns if col not in ['categorie', 'id']]
cols += ['categorie', 'id']
all_aligned_data = all_aligned_data[cols]"""

"# Réorganiser les colonnes pour s'assurer que 'categorie' et 'id' sont à la fin\ncols = [col for col in all_aligned_data.columns if col not in ['categorie', 'id']]\ncols += ['categorie', 'id']\nall_aligned_data = all_aligned_data[cols]"

In [8]:
#all_aligned_data = move_columns_to_end(all_aligned_data, ['id', 'categorie'])

In [7]:
all_aligned_data.shape

(172000, 754)

In [8]:
def segment_data(df, window_size=100, step_size=50, target_column='categorie'):
  # Validation des paramètres
  if window_size <= 0 or step_size <= 0:
    raise ValueError("Les paramètres 'window_size' et 'step_size' doivent être positifs.")
  if target_column not in df.columns:
    raise KeyError(f"La colonne cible '{target_column}' n'existe pas dans les données.")
  if len(df) < window_size:
    raise ValueError("La taille des données est inférieure à 'window_size'.")

  # Séparer les colonnes numériques et la colonne cible
  target = df[target_column]
  df_numeric = df.drop(columns=[target_column, 'id'])

  # Initialisation des segments et des cibles
  segments = []
  targets = []
   # Segmentation des données
  for start in range(0, len(df_numeric) - window_size + 1, step_size):
    # Fenêtre de données
    window = df_numeric.iloc[start:start + window_size].values
    segments.append(window)
    # Cible associée à la fenêtre (valeur de la dernière ligne)
    targets.append(int(target.iloc[start + window_size - 1]))

  # Conversion en tableaux numpy
  segments_array = np.array(segments)
  targets_array = np.array(targets)

  return segments_array, targets_array

In [51]:
"""print("imu aligned types : ", imu_aligned.dtypes)
print("emg aligned types : ", emg_aligned.dtypes)
print("ips aligned types : ", ips_aligned.dtypes)"""

'print("imu aligned types : ", imu_aligned.dtypes)\nprint("emg aligned types : ", emg_aligned.dtypes)\nprint("ips aligned types : ", ips_aligned.dtypes)'

In [56]:
"""print("emg procces nulls : ", emg_aligned.isnull().sum())
print("imu procces nulls : ", imu_aligned.isnull().sum())
print("ips procces nulls : ", ips_aligned.isnull().sum())"""

'print("emg procces nulls : ", emg_aligned.isnull().sum())\nprint("imu procces nulls : ", imu_aligned.isnull().sum())\nprint("ips procces nulls : ", ips_aligned.isnull().sum())'

In [41]:
# la segmentation pour chaque modalities
segmented_dataImu, target_Imu = segment_data(imu_aligned, window_size=50, step_size=25)
segmented_dataEmg, target_Emg = segment_data(emg_aligned, window_size=50, step_size=25)
segmented_dataIps, target_Ips = segment_data(ips_aligned, window_size=50, step_size=25)

In [42]:
print("non segmenté Imu : ", imu_aligned.shape)
print("non segmenté Emg : ", emg_aligned.shape)
print("non segmenté ips : ", ips_aligned.shape)
print("segmented data Imu : ", segmented_dataImu.shape)
print("segmented data Emg : ", segmented_dataEmg.shape)
print("segmented data ips : ", segmented_dataIps.shape)

non segmenté Imu :  (172000, 56)
non segmenté Emg :  (172000, 18)
non segmenté ips :  (172000, 684)
segmented data Imu :  (1719, 200, 54)
segmented data Emg :  (1719, 200, 16)
segmented data ips :  (1719, 200, 682)


In [43]:
print("segmented data Imu : ", target_Imu.shape)
print("segmented data Emg : ", target_Emg.shape)
print("segmented data ips : ", target_Ips.shape)

segmented data Imu :  (1719,)
segmented data Emg :  (1719,)
segmented data ips :  (1719,)


In [20]:
"""segmented_dataEmg = pd.DataFrame(segmented_dataEmg[0])
segmented_dataEmg.head(3)"""

'segmented_dataEmg = pd.DataFrame(segmented_dataEmg[0])\nsegmented_dataEmg.head(3)'

In [125]:
segmented_all_data, target_all_data = segment_data(all_aligned_data, window_size=12, step_size=6)

In [126]:
print("not segmented all_data : ", all_aligned_data.shape)
print("segmented all_data : ", segmented_all_data.shape)

not segmented all_data :  (172000, 754)
segmented all_data :  (28665, 12, 752)


In [67]:
"""# Mapping des colonnes
#column_names = list(all_aligned_data.columns) + ['categorie', 'id']
column_names = list(all_aligned_data.columns)
column_indices = {name: idx for idx, name in enumerate(column_names)}"""

"# Mapping des colonnes\n#column_names = list(all_aligned_data.columns) + ['categorie', 'id']\ncolumn_names = list(all_aligned_data.columns)\ncolumn_indices = {name: idx for idx, name in enumerate(column_names)}"

In [68]:
"""print("Colonnes réelles dans segmented_all_aligned_data :", segmented_all_data[0, 0, :])
"""

'print("Colonnes réelles dans segmented_all_aligned_data :", segmented_all_data[0, 0, :])\n'

In [71]:
"""print("Shape :", segmented_all_data.shape)
print("Exemple de données :", segmented_all_data[0, :, :])"""

'print("Shape :", segmented_all_data.shape)\nprint("Exemple de données :", segmented_all_data[0, :, :])'

In [72]:
"""print("noms des colonnes :", column_names)
print("indices des colonnes :", column_indices)"""

'print("noms des colonnes :", column_names)\nprint("indices des colonnes :", column_indices)'

In [111]:
print("Valeurs uniques dans la colonne 'categorie' avant toute transformation :")
print(np.unique(target_all_data))

Valeurs uniques dans la colonne 'categorie' avant toute transformation :
[0 1 2 3]


In [127]:
from sklearn.model_selection import train_test_split

# Architecture unifiée
X_unified = segmented_all_data
y_unified = target_all_data

In [128]:
print("Shape de X :", X_unified.shape)  # (n_samples, timesteps, features)
print("Shape de y :", y_unified.shape)  # (n_samples,)

Shape de X : (28665, 12, 752)
Shape de y : (28665,)


In [129]:
import numpy as np
print("Valeurs uniques dans y :", np.unique(y_unified))

Valeurs uniques dans y : [0 1 2 3]


In [66]:
print("Nombre de classes uniques :", len(np.unique(target_all_data)))

Nombre de classes uniques : 4


In [130]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, BatchNormalization

model_unified = Sequential()

# Couche CNN
model_unified.add(Conv1D(filters=55, kernel_size=3, activation='relu', input_shape=(X_unified.shape[1], X_unified.shape[2])))
model_unified.add(BatchNormalization())  # Ajout de BatchNorm pour stabiliser l'entraînement
model_unified.add(MaxPooling1D(pool_size=2))
model_unified.add(Dropout(0.7))

# Couche LSTM
model_unified.add(LSTM(50, activation='tanh', return_sequences=True))
model_unified.add(Dropout(0.7))
model_unified.add(LSTM(25, activation='tanh'))

# Couche Dense
model_unified.add(Dense(50, activation='relu'))
model_unified.add(Dropout(0.7))
model_unified.add(Dense(len(np.unique(y_unified)), activation='softmax'))

In [131]:
# Compilation
model_unified.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [132]:
# Résumé
model_unified.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_10 (Conv1D)          (None, 10, 55)            124135    
                                                                 
 batch_normalization_10 (Ba  (None, 10, 55)            220       
 tchNormalization)                                               
                                                                 
 max_pooling1d_10 (MaxPooli  (None, 5, 55)             0         
 ng1D)                                                           
                                                                 
 dropout_13 (Dropout)        (None, 5, 55)             0         
                                                                 
 lstm_19 (LSTM)              (None, 5, 50)             21200     
                                                                 
 dropout_14 (Dropout)        (None, 5, 50)           

In [139]:
!pip install pydot
!pip install graphviz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 1.7 MB/s eta 0:00:00


In [140]:
keras.utils.plot_model(model_unified)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [133]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_unified, y_unified, test_size=0.2, random_state=42)

In [100]:
"""from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)"""

'from tensorflow.keras.utils import to_categorical\n\ny_train = to_categorical(y_train)\ny_test = to_categorical(y_test)'

In [121]:
print("X_train NaN :", np.isnan(X_train).any())
print("X_test NaN :", np.isnan(X_test).any())
print("X_train inf :", np.isinf(X_train).any())
print("X_test inf :", np.isinf(X_test).any())
print("X_train max :", np.max(X_train), "X_train min :", np.min(X_train))

X_train NaN : False
X_test NaN : False
X_train inf : False
X_test inf : False
X_train max : 20.88246093720473 X_train min : -22.45843606673876


In [102]:
print("y_train max :", np.max(y_train), "y_train min :", np.min(y_train))
print("y_test max :", np.max(y_test), "y_test min :", np.min(y_test))

y_train max : 3 y_train min : 0
y_test max : 3 y_test min : 0


In [59]:
"""from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)"""

'from sklearn.preprocessing import StandardScaler\nscaler = StandardScaler()\nX_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)\nX_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)'

In [134]:
#history = model_unified.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))
history = model_unified.fit(X_train, y_train, epochs=20)

Epoch 1/20
717/717 [==============================] - 13s 13ms/step - loss: 0.2076 - accuracy: 0.9279
Epoch 2/20
717/717 [==============================] - 9s 12ms/step - loss: 0.0323 - accuracy: 0.9899
Epoch 3/20
717/717 [==============================] - 8s 12ms/step - loss: 0.0257 - accuracy: 0.9925
Epoch 4/20
717/717 [==============================] - 9s 12ms/step - loss: 0.0193 - accuracy: 0.9928
Epoch 5/20
717/717 [==============================] - 9s 12ms/step - loss: 0.0220 - accuracy: 0.9927
Epoch 6/20
717/717 [==============================] - 9s 12ms/step - loss: 0.0179 - accuracy: 0.9936
Epoch 7/20
717/717 [==============================] - 9s 12ms/step - loss: 0.0184 - accuracy: 0.9939
Epoch 8/20
717/717 [==============================] - 9s 12ms/step - loss: 0.0164 - accuracy: 0.9951
Epoch 9/20
717/717 [==============================] - 9s 12ms/step - loss: 0.0167 - accuracy: 0.9947
Epoch 10/20
717/717 [==============================] - 9s 12ms/step - loss: 0.0149 - accur

In [135]:
loss, accuracy = model_unified.evaluate(X_test, y_test)
print(f"Unified Model - Test Accuracy: {accuracy:.2f}")

180/180 [==============================] - 1s 4ms/step - loss: 0.0059 - accuracy: 0.9998
Unified Model - Test Accuracy: 1.00


In [105]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score, accuracy_score, classification_report

In [106]:
predictions = model_unified.predict(X_test)

22/22 [==============================] - 1s 26ms/step


In [107]:
predictions = np.argmax(predictions, axis=-1)

In [108]:
print("Accuracy : ", accuracy_score(y_test, predictions))
print("F1 score : ", f1_score(y_test, predictions, average='weighted'))
print("classification rapport : \n ", classification_report(y_test, predictions))

Accuracy :  1.0
F1 score :  1.0
classification rapport : 
                precision    recall  f1-score   support

           0       1.00      1.00      1.00       200
           1       1.00      1.00      1.00       161
           2       1.00      1.00      1.00       162
           3       1.00      1.00      1.00       165

    accuracy                           1.00       688
   macro avg       1.00      1.00      1.00       688
weighted avg       1.00      1.00      1.00       688



In [ ]:
from sklearn.model_selection import KFold, cross_val_score

# Supposons que X et y soient tes données et tes labels respectifs
kf = KFold(n_splits=5)
model = ...  # Défini ton modèle ici

scores = cross_val_score(model, X, y, cv=kf, scoring='accuracy')
print("Scores de validation croisée :", scores)
print("Précision moyenne :", scores.mean())


In [ ]:
# Architecture multi-modale : données séparées
X_ips = segmented_dataIps.drop(columns=['categorie', 'id']).values
X_imu = segmented_dataImu.drop(columns=['categorie', 'id']).values
X_emg = segmented_dataEmg.drop(columns=['categorie', 'id']).values
y_multimodal = segmented_dataEmg['categorie'].values

# Séparer les données en train/test pour les deux architectures
X_unified_train, X_unified_test, y_unified_train, y_unified_test = train_test_split(X_unified, y_unified, test_size=0.2, random_state=42)

X_ips_train, X_ips_test, y_ips_train, y_ips_test = train_test_split(X_ips, y_multimodal, test_size=0.2, random_state=42)
X_imu_train, X_imu_test, y_imu_train, y_imu_test = train_test_split(X_imu, y_multimodal, test_size=0.2, random_state=42)
X_emg_train, X_emg_test, y_emg_train, y_emg_test = train_test_split(X_emg, y_multimodal, test_size=0.2, random_state=42)

In [ ]:
print("Forme des données unifiées :", X_unified.shape)
print("Forme des données IPS :", X_ips.shape)
print("Forme des données IMU :", X_imu.shape)
print("Forme des données EMG :", X_emg.shape)


NameError: name 'X_unified' is not defined

In [ ]:
# Reshape pour CNN/LSTM
X_unified_train = X_unified_train.reshape(X_unified_train.shape[0], 1, X_unified_train.shape[1])
X_unified_test = X_unified_test.reshape(X_unified_test.shape[0], 1, X_unified_test.shape[1])


In [ ]:
!pip install tensorflow

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, LSTM, Flatten, Dropout

# Modèle CNN + LSTM pour architecture unifiée
model_unified = Sequential()

# Couche CNN
model_unified.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_unified_train.shape[1], X_unified_train.shape[2])))
model_unified.add(MaxPooling1D(pool_size=2))
model_unified.add(Dropout(0.2))

# Couche LSTM
model_unified.add(LSTM(50, activation='relu', return_sequences=True))
model_unified.add(LSTM(50, activation='relu'))

# Couche Dense finale
model_unified.add(Dense(100, activation='relu'))
model_unified.add(Dense(len(set(y_unified)), activation='softmax'))  # Softmax pour classification

# Compilation
model_unified.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Résumé du modèle
model_unified.summary()


In [ ]:
"""def align_data_to_median(modalities):

  # Obtenir le nombre de fenêtres pour chaque modalité
  window_counts = [modality.shape[0] for modality in modalities]
  # Calculer la médiane des nombres de fenêtres
  median_windows = int(np.median(window_counts))
  print(f"Médiane des fenêtres : {median_windows}")

  aligned_modalities = []
  for modality in modalities:
      num_windows = modality.shape[0]
      if num_windows > median_windows:
          # Tronquer si le nombre de fenêtres est supérieur à la médiane
          aligned_modality = modality[:median_windows]
      elif num_windows < median_windows:
          # Compléter si le nombre de fenêtres est inférieur à la médiane
          padding_needed = median_windows - num_windows
          # Répéter les dernières fenêtres pour compléter
          padding = np.tile(modality[-1:], (padding_needed, 1, 1))
          aligned_modality = np.concatenate([modality, padding], axis=0)
      else:
          # Pas de changement si le nombre de fenêtres est égal à la médiane
          aligned_modality = modality
      aligned_modalities.append(aligned_modality)

  return aligned_modalities"""

'def align_data_to_median(modalities):\n\n  # Obtenir le nombre de fenêtres pour chaque modalité\n  window_counts = [modality.shape[0] for modality in modalities]\n  # Calculer la médiane des nombres de fenêtres\n  median_windows = int(np.median(window_counts))\n  print(f"Médiane des fenêtres : {median_windows}")\n\n  aligned_modalities = []\n  for modality in modalities:\n      num_windows = modality.shape[0]\n      if num_windows > median_windows:\n          # Tronquer si le nombre de fenêtres est supérieur à la médiane\n          aligned_modality = modality[:median_windows]\n      elif num_windows < median_windows:\n          # Compléter si le nombre de fenêtres est inférieur à la médiane\n          padding_needed = median_windows - num_windows\n          # Répéter les dernières fenêtres pour compléter\n          padding = np.tile(modality[-1:], (padding_needed, 1, 1))\n          aligned_modality = np.concatenate([modality, padding], axis=0)\n      else:\n          # Pas de changeme

In [ ]:
"""# Données segmentées
modalities = [segmented_dataImu, segmented_dataEmg, segmented_dataIps]

# Aligner les données à la médiane
aligned_modalities = align_data_to_median(modalities)

# Vérifier les dimensions après alignement
for i, aligned_modality in enumerate(aligned_modalities):
    print(f"Modalité {i + 1} après alignement : {aligned_modality.shape}")

"""

In [ ]:
"""merged_results = []

for participant_id in normalized_dataEmg['id'].unique():
    emg_participant = normalized_dataEmg[normalized_dataEmg['id'] == participant_id]
    imu_participant = normalized_dataImu[normalized_dataImu['id'] == participant_id]
    ips_participant = normalized_dataIps[normalized_dataIps['id'] == participant_id]

    # Fusionner pour le participant actuel
    merged_participant = merge_modalities(emg_participant, imu_participant, ips_participant)

    # Sauvegarder les résultats pour éviter de surcharger la RAM
    merged_results.append(merged_participant)

    # Optionnel : Sauvegarder temporairement sur disque pour libérer de la mémoire
    merged_participant.to_csv(f"merged_participant_{participant_id}.csv", index=False)
"""

In [ ]:
"""1 -> back -> ips_1.csv , emg_1.csv, imu_1.csv
1 -> forward -> ips_1.csv , emg_1.csv, imu_1.csv
1 -> halfsquat -> ips_1.csv , emg_1.csv, imu_1.csv
1 -> still -> ips_1.csv , emg_1.csv, imu_1.csv
2 -> back -> ips_1.csv , emg_1.csv, imu_1.csv
2 -> forward -> ips_1.csv , emg_1.csv, imu_1.csv
2 -> halfsquat -> ips_1.csv , emg_1.csv, imu_1.csv
2 -> still -> ips_1.csv , emg_1.csv, imu_1.csv
3 -> back -> ips_1.csv , emg_1.csv, imu_1.csv
3 -> forward -> ips_1.csv , emg_1.csv, imu_1.csv
3 -> halfsquat -> ips_1.csv , emg_1.csv, imu_1.csv
3 -> still -> ips_1.csv , emg_1.csv, imu_1.csv
...
25 -> back -> ips_1.csv , emg_1.csv, imu_1.csv
25 -> forward -> ips_1.csv , emg_1.csv, imu_1.csv
25 -> halfsquat -> ips_1.csv , emg_1.csv, imu_1.csv
25 -> still -> ips_1.csv , emg_1.csv, imu_1.csv"""
